# SITCOMTN-082: Hard Point Breakaway Analysis

Notebook containing data analysis for the [LVV-T231] test case.  
https://jira.lsstcorp.org/browse/SITCOM-838
The script used to run this test case can be found in [M13T004.py].  

[LVV-T231]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T231
[M13T004.py]: https://github.com/lsst-ts/ts_m1m3supporttesting/blob/develop/M13T004.py
[Technote]: https://sitcomtn-082.lsst.io/


## Summary

1. The analysis compares the measured hardpoint stiffness with specified stiffness.
2. This notebook analyses the offline data produced by the script [M13T004.py] and stored on m1m3-dev.cp.lsst.org.
3. The data show the forces changing with hardpoint extension and compression.
4. Compression is marked by a "-" in front of "-999999999"in the file name.

## Expected Results

It is expected that the fitted hardpoint stiffness is within the specified stiffness. 

## Requirement 

1. Create the plots for the data of the hard point breakaway test from 2023-03-10 and 2022-11-07 and newer data.
2. Use the attached plot as an example.
3. Add the slope for the specification (100N/um)
4. Transform the X axis into um.
5. Update to use the data from the EFD.
6. Update to use sequence numbers as produced by the SAL scripts.
7. Update to not only show one repetition of compression or extension in a diagram (usually, there are three of them).


## Prepare notebook

### Prepare notebook for test without Block (May 2023)

This is the list of Hardpoint Breakaway Tests run using SAL script. You can pick one of the test. 
| Elevation <br> (deg)| Azimuth <br> (deg) | t start <br> (YYYY-MM-DDTHH:MM:SS) | SALIndex | 
| --- | --- | --- | --- |
| 0 | -29.69 | 2023-05-30T21:26:51 | 100056 |
| 1 | -29.69 | 2023-05-30T22:40:34 | 100057 |
| 5 | -29.69 | 2023-05-31T00:00:10 | 100058 |
| 10 | -29.69 | 2023-05-31T01:03:26 | 100059 |
| 20 | 153 | 2023-05-27T02:49:55 | 100036 |
| 20 | 153 | 2023-05-30T08:26:34 | 100047 |
| 40 | 153 | 2023-05-26T02:23:28 | 100034 |
| 89.95 | 153 | 2023-06-20T03:11:00 | 100038 |
| 90 | -29.69 | 2023-05-31T05:44:14 | 100061 |

In [ ]:
# Number of hard points in M1M3
number_of_hardpoints = 6

# scale factor for "encoder"
scale = 0.2442

# m to micron.
m_to_um = 1000000

# number of points around zero force for linear fit
n_points = 100

# Specified stiffnest from the requirements
spect_stiffness = 100

# Kernel size for edge detection
kernel_size = 100

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black
%matplotlib inline

In [ ]:
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy import special

import scipy.optimize as opt
from astropy.time import Time, TimeDelta
from scipy.signal import find_peaks
from lsst.sitcom import vandv

from matplotlib.lines import Line2D

### Prepare notebook for test with Block (Nov-Dec 2023)

Block-61 and Block-147 are both for hardpoint breakaway test. 
- Block-61: Not clear for elevation.
- Block-147: El = 45 deg.
- Block-149: El = 30 deg.

In [ ]:
blocks = [61, 147, 149]

In [ ]:
from lsst.summit.utils.blockUtils import BlockParser
from datetime import datetime, timedelta


dayObs_start = 20231122
dayObs_end = 20231207
# Create instances for dates from start_date to end_date

start_date = Time(datetime.strptime(str(dayObs_start), "%Y%m%d"))
end_date = Time(datetime.strptime(str(dayObs_end), "%Y%m%d"))

# Create instances for dates from start_date to end_date
current_date = start_date

block_id = []
block_start = []
block_end = []

for block in blocks:
    current_date = start_date
    while current_date <= end_date:
        block_parser = BlockParser(current_date.datetime.strftime("%Y%m%d"))
        seqNums = block_parser.getSeqNums(block)
        print(current_date)
        if len(seqNums) > 0:
            for seqNum in seqNums:
                info = block_parser.getBlockInfo(block, seqNum)
                print(info)
                if info.states[-1].state == 8:
                    time_start = info.begin.iso
                    time_end = info.end.iso
                    block_id.append(block)
                    block_start.append(time_start)
                    block_end.append(time_end)
        current_date += TimeDelta(1.0, format="jd")  # 'jd' stands for Julian days

block_id, block_start, block_end

In [ ]:
# El =  89.95 ; Az= 153
# t_start= "2023-06-20T03:11:00"
# sal_index = 100038

## Query the data

In [ ]:
client = vandv.efd.create_efd_client()

In [ ]:
import tabulate

data = [block_id, block_start, block_end]
header = ["Block ID", "Start Time", "End Time"]

# Creating a DataFrame
df = pd.DataFrame(
    {"Block ID": block_id, "Start Time": block_start, "End Time": block_end}
)
# Displaying the pivoted table using tabulate
table = tabulate.tabulate(df, headers="keys", tablefmt="html", showindex=True)

table

In [ ]:
# Put the row number of a block to check the results.
index = -1

In [ ]:
block_n = block_id[index]
t_start = Time(block_start[index], format="iso", scale="utc")
t_end = Time(block_end[index], format="iso", scale="utc")

In [ ]:
print(t_start, t_end)

In [ ]:
# Query Elvation and Azmuth.
el = await client.select_time_series(
    "lsst.sal.MTMount.elevation",
    ["actualPosition"],
    t_start,
    t_end,
)
az = await client.select_time_series(
    "lsst.sal.MTMount.azimuth",
    ["actualPosition"],
    t_start,
    t_end,
)

El = round(el["actualPosition"][-1])
Az = round(az["actualPosition"][-1])

In [ ]:
df_script = await client.select_time_series(
    "lsst.sal.Script.logevent_logMessage",
    ["salIndex", "level", "message"],
    t_start,
    t_end,
)

# Select messages with the associated SAL Index
# df_script = df_script[df_script["salIndex"] == sal_index]

In [ ]:
df_hp = await client.select_time_series(
    "lsst.sal.MTM1M3.hardpointActuatorData",
    "*",
    Time(df_script.index[0]),  # Tests start
    Time(df_script.index[-1]),  # Tests end
)

In [ ]:
df_hp_status = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_hardpointTestStatus",
    "*",
    Time(df_script.index[0]),  # Tests start
    Time(df_script.index[-1]),  # Tests end
)

## Analyze the data

Let's start analyzing the data by having a look at the measured forces over time.  
This will help identifying any potential issues with one of the hard points right away. 

In [ ]:
fig, axs = plt.subplots(number_of_hardpoints, 1, figsize=(13, 20), dpi=96)

for hp_index in range(number_of_hardpoints):
    handles = []
    df_script_sub = df_script[
        df_script["message"].str.contains(f"Hard point {hp_index + 1} test")
    ]
    df = df_hp.loc[df_script_sub.index[0] : df_script_sub.index[-1]]
    # hp_test = df_hp[df_hp["hardpointActuator"] == hp_index + 1]

    # Create plot axes
    # fig, axs = plt.subplots(num=f"hp_timeline_{hp_index}", figsize=(10, 3), dpi=96)

    # Show the data
    axs[hp_index].plot(df[f"measuredForce{hp_index}"], zorder=5, color="black")

    # enumerations for HardpointTest.
    MovingNegative = df_hp_status[
        df_hp_status[f"testState{hp_index}"] == 2
    ].index.values
    TestingPositive = df_hp_status[
        df_hp_status[f"testState{hp_index}"] == 3
    ].index.values
    TestingNegative = df_hp_status[
        df_hp_status[f"testState{hp_index}"] == 4
    ].index.values
    MovingReference = df_hp_status[
        df_hp_status[f"testState{hp_index}"] == 5
    ].index.values

    # Add some props to the plot
    axs[hp_index].grid("-", lw=0.5, alpha=0.25)
    axs[hp_index].set_ylabel(f"HP{hp_index + 1} Measured\n Forces [N]")
    axs[hp_index].set_xlabel("Time [UTC]")

    # Format x-axis with time
    axs[hp_index].xaxis.set_major_locator(mdates.MinuteLocator())
    axs[hp_index].xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))

    # Clean up the plot
    axs[hp_index].spines["top"].set_visible(False)
    axs[hp_index].spines["right"].set_visible(False)
    comp_min = 2981
    comp_max = 3959
    tens_min = -4420
    tens_max = -3456

    # Seperate the status of the hard points
    axs[hp_index].axvline(MovingNegative, color="k", linestyle=":")
    axs[hp_index].axvline(TestingPositive, color="k", linestyle=":")
    axs[hp_index].axvline(TestingNegative, color="k", linestyle=":")
    axs[hp_index].axvline(MovingReference, color="k", linestyle=":")

    axs[hp_index].axhspan(comp_min, comp_max, color="grey", alpha=0.2)
    axs[hp_index].axhspan(tens_min, tens_max, color="yellow", alpha=0.2)

    # Input some texts on the plots

    axs[hp_index].text(
        0.05, 0.875, "Compression", transform=axs[hp_index].transAxes, weight="bold"
    )
    axs[hp_index].text(
        0.05, 0.065, "Tension", transform=axs[hp_index].transAxes, weight="bold"
    )
    axs[hp_index].text(
        1.00,
        1.100,
        "BreakAway Range:",
        transform=axs[hp_index].transAxes,
        weight="bold",
    )
    axs[hp_index].text(
        1.00, 0.820, f"{comp_min}" + "N", transform=axs[hp_index].transAxes
    )
    axs[hp_index].text(
        1.00, 0.970, f"{comp_max}" + "N", transform=axs[hp_index].transAxes
    )
    axs[hp_index].text(
        1.00, 0.020, f"{tens_min}" + "N", transform=axs[hp_index].transAxes
    )
    axs[hp_index].text(
        1.00, 0.170, f"{tens_max}" + "N", transform=axs[hp_index].transAxes
    )

    # Define the region of 1) Moving HP in the Negative Force 2) Testing HP on Positive Force 3) Testing HP on Negative Force.

    MovingNegative_HP = df[f"measuredForce{hp_index}"][
        (df.index.values > MovingNegative[0]) & (df.index.values < TestingPositive[0])
    ]
    TestingPositive_HP = df[f"measuredForce{hp_index}"][
        (df.index.values > TestingPositive[0]) & (df.index.values < TestingNegative[0])
    ]
    TestingNegative_HP = df[f"measuredForce{hp_index}"][
        (df.index.values > TestingNegative[0]) & (df.index.values < MovingReference[0])
    ]
    MovingReference_HP = df[f"measuredForce{hp_index}"][
        (df.index.values > MovingReference[0])
    ]

    axs[hp_index].plot(MovingNegative_HP, color="g", zorder=5)
    axs[hp_index].plot(TestingPositive_HP, color="blue", zorder=5)
    axs[hp_index].plot(TestingNegative_HP, color="r", zorder=5)
    axs[hp_index].plot(MovingReference_HP, color="grey", zorder=5)

    line1 = Line2D([0], [0], label="Moving Negative", color="g")
    line2 = Line2D([0], [0], label="Testing Positive", color="b")
    line3 = Line2D([0], [0], label="Testing Negative ", color="r")

    handles.extend([line1, line2, line3])

    # axs[hp_index].legend(handles=handles,fontsize=11,framealpha=1,bbox_to_anchor=(1.2, 0.8))
    axs[hp_index].legend(handles=handles, fontsize=11, framealpha=1)

    axs[hp_index].set_title(
        f"Individual Hard Points Breakaway Test at El:{El} deg, Az:{Az} deg\n"
        f"HP{hp_index+1} - Block-{block_n} - {df.index[0].strftime('%Y-%m-%d')}"
    )
    # fig.autofmt_xdate()
    fig.show()


os.makedirs("plots", exist_ok=True)
fig.tight_layout(pad=2.0)
fig.savefig(f"plots/m1m3004_hp_timeline_El_{El}.png", dpi=150, bbox_inches="tight")

In [ ]:
# fig, axs = plt.subplots(number_of_hardpoints, 1, figsize=(13, 20), dpi=96)
for hp_index in range(number_of_hardpoints):
    fig, axs = plt.subplots(1, 1, figsize=(8, 5), dpi=96)
    handles = []
    df_script_sub = df_script[
        df_script["message"].str.contains(f"Hard point {hp_index + 1} test")
    ]
    df = df_hp.loc[df_script_sub.index[0] : df_script_sub.index[-1]]
    # hp_test = df_hp[df_hp["hardpointActuator"] == hp_index + 1]

    # Create plot axes
    # fig, axs = plt.subplots(num=f"hp_timeline_{hp_index}", figsize=(10, 3), dpi=96)

    # Show the data
    axs.plot(df[f"measuredForce{hp_index}"], zorder=5, color="black")

    # enumerations for HardpointTest.
    MovingNegative = df_hp_status[
        df_hp_status[f"testState{hp_index}"] == 2
    ].index.values
    TestingPositive = df_hp_status[
        df_hp_status[f"testState{hp_index}"] == 3
    ].index.values
    TestingNegative = df_hp_status[
        df_hp_status[f"testState{hp_index}"] == 4
    ].index.values
    MovingReference = df_hp_status[
        df_hp_status[f"testState{hp_index}"] == 5
    ].index.values

    # Add some props to the plot
    axs.grid("-", lw=0.5, alpha=0.25)
    axs.set_ylabel(f"HP{hp_index + 1} Measured\n Forces [N]")
    axs.set_xlabel("Time [UTC]")

    # Format x-axis with time
    axs.xaxis.set_major_locator(mdates.MinuteLocator())
    axs.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))

    # Clean up the plot
    axs.spines["top"].set_visible(False)
    axs.spines["right"].set_visible(False)
    comp_min = 2981
    comp_max = 3959
    tens_min = -4420
    tens_max = -3456

    # Seperate the status of the hard points
    axs.axvline(MovingNegative, color="k", linestyle=":")
    axs.axvline(TestingPositive, color="k", linestyle=":")
    axs.axvline(TestingNegative, color="k", linestyle=":")
    axs.axvline(MovingReference, color="k", linestyle=":")

    axs.axhspan(comp_min, comp_max, color="grey", alpha=0.2)
    axs.axhspan(tens_min, tens_max, color="yellow", alpha=0.2)

    # Input some texts on the plots

    axs.text(0.05, 0.875, "Compression", transform=axs.transAxes, weight="bold")
    axs.text(0.05, 0.065, "Tension", transform=axs.transAxes, weight="bold")
    axs.text(0.70, 0.970, "BreakAway Range:", transform=axs.transAxes, weight="bold")
    axs.text(1.00, 0.820, f"{comp_min}" + "N", transform=axs.transAxes)
    axs.text(1.00, 0.970, f"{comp_max}" + "N", transform=axs.transAxes)
    axs.text(1.00, 0.020, f"{tens_min}" + "N", transform=axs.transAxes)
    axs.text(1.00, 0.170, f"{tens_max}" + "N", transform=axs.transAxes)

    # Define the region of 1) Moving HP in the Negative Force 2) Testing HP on Positive Force 3) Testing HP on Negative Force.

    MovingNegative_HP = df[f"measuredForce{hp_index}"][
        (df.index.values > MovingNegative[0]) & (df.index.values < TestingPositive[0])
    ]
    TestingPositive_HP = df[f"measuredForce{hp_index}"][
        (df.index.values > TestingPositive[0]) & (df.index.values < TestingNegative[0])
    ]
    TestingNegative_HP = df[f"measuredForce{hp_index}"][
        (df.index.values > TestingNegative[0]) & (df.index.values < MovingReference[0])
    ]
    MovingReference_HP = df[f"measuredForce{hp_index}"][
        (df.index.values > MovingReference[0])
    ]

    axs.plot(MovingNegative_HP, color="g", zorder=5)
    axs.plot(TestingPositive_HP, color="blue", zorder=5)
    axs.plot(TestingNegative_HP, color="r", zorder=5)
    axs.plot(MovingReference_HP, color="grey", zorder=5)

    line1 = Line2D([0], [0], label="Moving Negative", color="g")
    line2 = Line2D([0], [0], label="Testing Positive", color="b")
    line3 = Line2D([0], [0], label="Testing Negative ", color="r")

    handles.extend([line1, line2, line3])

    # axs[hp_index].legend(handles=handles,fontsize=11,framealpha=1,bbox_to_anchor=(1.2, 0.8))
    axs.legend(handles=handles, fontsize=11, framealpha=1, bbox_to_anchor=(0.9, 0.2))

    axs.set_title(
        f"Individual Hard Points Breakaway Test at El:{El} deg, Az:{Az} deg\n"
        f"HP{hp_index+1} - Block-{block_n} - {df.index[0].strftime('%Y-%m-%d')}"
    )
    # fig.autofmt_xdate()

    os.makedirs("plots", exist_ok=True)
    fig.tight_layout(pad=2.0)
    fig.savefig(
        f"plots/m1m3004_hp_timeline_El_{El}_{hp_index+1}_Block_{block_n}.png",
        dpi=150,
        bbox_inches="tight",
    )
    fig.show()


Now plot the hystersis for each hardpoint. <br> 

1. Plot HP measured forces for each moving status with respect to displacement. <br>
2. In order to compare the shape of hystersis, zero point of each phase is shifted to displacement= 0.  <br> 
3. Then, find the fitted hardpoint stiffness - find the linear slopes at F=0. Add the slope for the specification (100N/um) <br>



In [ ]:
for hp_index in range(number_of_hardpoints):
    fig, axs = plt.subplots(1, 1, figsize=(10, 6), dpi=96)
    handles = []

    df_script_sub = df_script[
        df_script["message"].str.contains(f"Hard point {hp_index + 1} test")
    ]
    df = df_hp.loc[df_script_sub.index[0] : df_script_sub.index[-1]]

    MovingNegative = df_hp_status[
        df_hp_status[f"testState{hp_index}"] == 2
    ].index.values
    TestingPositive = df_hp_status[
        df_hp_status[f"testState{hp_index}"] == 3
    ].index.values
    TestingNegative = df_hp_status[
        df_hp_status[f"testState{hp_index}"] == 4
    ].index.values
    MovingReference = df_hp_status[
        df_hp_status[f"testState{hp_index}"] == 5
    ].index.values

    MovingNegative_HP = df[f"measuredForce{hp_index}"][
        (df.index.values > MovingNegative[0]) & (df.index.values < TestingPositive[0])
    ]
    TestingPositive_HP = df[f"measuredForce{hp_index}"][
        (df.index.values > TestingPositive[0]) & (df.index.values < TestingNegative[0])
    ]
    TestingNegative_HP = df[f"measuredForce{hp_index}"][
        (df.index.values > TestingNegative[0]) & (df.index.values < MovingReference[0])
    ]
    MovingReference_HP = df[f"measuredForce{hp_index}"][
        (df.index.values > MovingReference[0])
    ]

    MovingNegative_displacement_HP = (
        1000000
        * df[f"displacement{hp_index}"][
            (df.index.values > MovingNegative[0])
            & (df.index.values < TestingPositive[0])
        ]
    )
    TestingPositive_displacement_HP = (
        1000000
        * df[f"displacement{hp_index}"][
            (df.index.values > TestingPositive[0])
            & (df.index.values < TestingNegative[0])
        ]
    )
    TestingNegative_displacement_HP = (
        1000000
        * df[f"displacement{hp_index}"][
            (df.index.values > TestingNegative[0])
            & (df.index.values < MovingReference[0])
        ]
    )
    MovingReference_displacement_HP = (
        1000000 * df[f"displacement{hp_index}"][(df.index.values > MovingReference[0])]
    )

    zero_force_point_MN = MovingNegative_HP.index.get_loc(
        MovingNegative_HP.abs().idxmin()
    )
    zero_force_point_TP = TestingPositive_HP.index.get_loc(
        TestingPositive_HP.abs().idxmin()
    )
    zero_force_point_TN = TestingNegative_HP.index.get_loc(
        TestingNegative_HP.abs().idxmin()
    )

    x = np.arange(800) - 400

    if zero_force_point_MN > 10 and (max(TestingPositive_HP) > 0):
        min_MN = min(zero_force_point_MN, 10)
        m_MN, b_MN = np.polyfit(
            MovingNegative_displacement_HP[
                zero_force_point_MN - min_MN : zero_force_point_MN + min_MN
            ],
            MovingNegative_HP[
                zero_force_point_MN - min_MN : zero_force_point_MN + min_MN
            ],
            1,
        )
        x_zero_MN = -b_MN / m_MN
        axs.plot(
            MovingNegative_displacement_HP - x_zero_MN,
            MovingNegative_HP,
            color="g",
            marker="o",
            markevery=50,
        )
        axs.plot(x, x * m_MN, color="g")
        axs.text(200, 1500, "Moving Negative= " + f"{m_MN:.3f}N/um")
        line1 = Line2D([0], [0], label="Moving Negative", color="g")

    if (zero_force_point_TP > 10) and (max(TestingPositive_HP) > 0):
        min_TP = min(zero_force_point_TP, 10)
        m_TP, b_TP = np.polyfit(
            TestingPositive_displacement_HP[
                zero_force_point_TP - min_TP : zero_force_point_TP + min_TP
            ],
            TestingPositive_HP[
                zero_force_point_TP - min_TP : zero_force_point_TP + min_TP
            ],
            1,
        )
        x_zero_TP = -b_TP / m_TP
        axs.plot(
            TestingPositive_displacement_HP - x_zero_TP,
            TestingPositive_HP,
            color="b",
            marker="o",
            markevery=50,
        )
        axs.plot(x, x * m_TP, color="b")
        axs.text(200, 1000, "Testing Positive= " + f"{m_TP:.3f}N/um")
        line2 = Line2D([0], [0], label="Testing Positive", color="b")

    if zero_force_point_TN > 10:
        min_TN = min(zero_force_point_TN, 10)
        m_TN, b_TN = np.polyfit(
            TestingNegative_displacement_HP[
                zero_force_point_TN - min_TN : zero_force_point_TN + min_TN
            ],
            TestingNegative_HP[
                zero_force_point_TN - min_TN : zero_force_point_TN + min_TN
            ],
            1,
        )
        x_zero_TN = -b_TN / m_TN
        axs.plot(
            TestingNegative_displacement_HP - x_zero_TN,
            TestingNegative_HP,
            color="r",
            marker="o",
            markevery=50,
        )
        axs.plot(x, x * m_TN, color="r")
        axs.text(200, 500, "Testing Negative= " + f"{m_TN:.3f}N/um")
        line3 = Line2D([0], [0], label="Testing Negative ", color="r")

    if zero_force_point_MN > 10 or zero_force_point_MN > 10 or zero_force_point_MN > 10:
        axs.plot(x, x * 100, ":", color="k")
        axs.text(200, 2000, "Stiffness:")
        line4 = Line2D(
            [0], [0], label="Specification (100N/${\mu}$m)", linestyle=":", color="k"
        )
        handles.extend([line1, line2, line3, line4])
        axs.legend(handles=handles, fontsize=13, framealpha=1, loc="lower right")

    # Add axis labels
    axs.set_xlabel("${\Delta}$displacement(${\mu}$m)")
    axs.set_ylabel("Force (N)")

    # Add grid and legend to plot
    axs.grid("-", lw=0.5, alpha=0.25)

    axs.spines["top"].set_visible(False)
    axs.spines["right"].set_visible(False)
    axs.xaxis.set_ticks_position("bottom")
    axs.set_xlim(-500, 500)
    axs.set_ylim(-6000, 6000)
    axs.axhspan(comp_min, comp_max, color="grey", alpha=0.2)
    axs.axhspan(tens_min, tens_max, color="yellow", alpha=0.2)

    axs.axhline(y=0, color="k")
    axs.axvline(x=0, color="k")

    axs.text(-450, np.mean([comp_min, comp_max]), "Compression", weight="bold")
    axs.text(-450, np.mean([tens_min, tens_max]), "Tension", weight="bold")

    # handles.extend([line1,line2, line3, line4])
    # axs[hp_index].legend(handles=handles,fontsize=11,framealpha=1)

    # Add title to plot
    title = (
        f"Individual Hard Point Breakaway Test at El:{El} deg, Az:{Az} deg\n"
        f"HP{hp_index + 1} - Block {block_n} -"
        f" {df.index[0].isoformat(timespec='seconds')[:-6]} -"
        f" {df.index[-1].isoformat(timespec='seconds')[:-6]}"
    )
    axs.set_title(f"{title}")

    # Save plot as png file
    os.makedirs("plots", exist_ok=True)
    filename = (
        f"Force_displacement_block_{block_n}_El_{El}_{hp_index}_Block_{block_n}.png"
    )
    fig.savefig(f"plots/{filename}", dpi=150, bbox_inches="tight")
    fig.show()

In [ ]:
for hp_index in range(number_of_hardpoints):
    handles = []
    fig, axs = plt.subplots(1, 2, figsize=(28, 14), dpi=96)

    df_script_sub = df_script[
        df_script["message"].str.contains(f"Hard point {hp_index + 1} test")
    ]
    df = df_hp.loc[df_script_sub.index[0] : df_script_sub.index[-1]]

    MovingNegative = df_hp_status[
        df_hp_status[f"testState{hp_index}"] == 2
    ].index.values
    TestingPositive = df_hp_status[
        df_hp_status[f"testState{hp_index}"] == 3
    ].index.values
    TestingNegative = df_hp_status[
        df_hp_status[f"testState{hp_index}"] == 4
    ].index.values
    MovingReference = df_hp_status[
        df_hp_status[f"testState{hp_index}"] == 5
    ].index.values

    MovingNegative_HP = df[f"measuredForce{hp_index}"][
        (df.index.values > MovingNegative[0]) & (df.index.values < TestingPositive[0])
    ]
    TestingPositive_HP = df[f"measuredForce{hp_index}"][
        (df.index.values > TestingPositive[0]) & (df.index.values < TestingNegative[0])
    ]
    TestingNegative_HP = df[f"measuredForce{hp_index}"][
        (df.index.values > TestingNegative[0]) & (df.index.values < MovingReference[0])
    ]
    MovingReference_HP = df[f"measuredForce{hp_index}"][
        (df.index.values > MovingReference[0])
    ]

    MovingNegative_displacement_HP = (
        1000000
        * df[f"displacement{hp_index}"][
            (df.index.values > MovingNegative[0])
            & (df.index.values < TestingPositive[0])
        ]
    )
    TestingPositive_displacement_HP = (
        1000000
        * df[f"displacement{hp_index}"][
            (df.index.values > TestingPositive[0])
            & (df.index.values < TestingNegative[0])
        ]
    )
    TestingNegative_displacement_HP = (
        1000000
        * df[f"displacement{hp_index}"][
            (df.index.values > TestingNegative[0])
            & (df.index.values < MovingReference[0])
        ]
    )
    MovingReference_displacement_HP = (
        1000000 * df[f"displacement{hp_index}"][(df.index.values > MovingReference[0])]
    )

    zero_force_point_MN = MovingNegative_HP.index.get_loc(
        MovingNegative_HP.abs().idxmin()
    )
    zero_force_point_TP = TestingPositive_HP.index.get_loc(
        TestingPositive_HP.abs().idxmin()
    )
    zero_force_point_TN = TestingNegative_HP.index.get_loc(
        TestingNegative_HP.abs().idxmin()
    )

    if (
        (zero_force_point_MN > 10)
        & (zero_force_point_TP > 10)
        & (zero_force_point_TN > 10)
    ):
        min_MN = min(zero_force_point_MN, 10)
        min_TP = min(zero_force_point_TP, 10)
        min_TN = min(zero_force_point_TN, 10)

        m_MN, b_MN = np.polyfit(
            MovingNegative_displacement_HP[
                zero_force_point_MN - min_MN : zero_force_point_MN + min_MN
            ],
            MovingNegative_HP[
                zero_force_point_MN - min_MN : zero_force_point_MN + min_MN
            ],
            1,
        )
        m_TP, b_TP = np.polyfit(
            TestingPositive_displacement_HP[
                zero_force_point_TP - min_TP : zero_force_point_TP + min_TP
            ],
            TestingPositive_HP[
                zero_force_point_TP - min_TP : zero_force_point_TP + min_TP
            ],
            1,
        )
        m_TN, b_TN = np.polyfit(
            TestingNegative_displacement_HP[
                zero_force_point_TN - min_TN : zero_force_point_TN + min_TN
            ],
            TestingNegative_HP[
                zero_force_point_TN - min_TN : zero_force_point_TN + min_TN
            ],
            1,
        )

        x_zero_MN = -b_MN / m_MN
        x_zero_TP = -b_TP / m_TP
        x_zero_TN = -b_TN / m_TN

        x = (np.arange(40) - 20) * 10

        axs[0].plot(
            TestingPositive_displacement_HP - x_zero_TP,
            TestingPositive_HP,
            color="b",
            marker="o",
            markevery=50,
        )
        axs[0].plot(
            TestingNegative_displacement_HP - x_zero_TN,
            TestingNegative_HP,
            color="r",
            marker="o",
            markevery=50,
        )

        line1 = Line2D([0], [0], label="Testing Positive", color="b")
        line2 = Line2D([0], [0], label="Testing Negative ", color="r")
        line3 = Line2D([0], [0], label="Residual - Testing Positive", color="skyblue")
        line4 = Line2D([0], [0], label="Residual - Testing Negative", color="purple")

        def erfunc(x, mFL, a, b):
            return mFL * special.erf((x - a) / (b * np.sqrt(2)))

        x_fit_pos_TP = (
            TestingPositive_displacement_HP[(TestingPositive_HP > 0)] - x_zero_TP
        )
        x_fit_neg_TP = (
            TestingPositive_displacement_HP[(TestingPositive_HP < 0)] - x_zero_TP
        )

        x_fit_pos_TN = (
            TestingNegative_displacement_HP[(TestingNegative_HP > 0)] - x_zero_TN
        )
        x_fit_neg_TN = (
            TestingNegative_displacement_HP[(TestingNegative_HP < 0)] - x_zero_TN
        )

        y_fit_pos_TP = TestingPositive_HP[(TestingPositive_HP > 0)]
        y_fit_neg_TP = TestingPositive_HP[(TestingPositive_HP < 0)]

        y_fit_pos_TN = TestingNegative_HP[(TestingNegative_HP > 0)]
        y_fit_neg_TN = TestingNegative_HP[(TestingNegative_HP < 0)]

        x_fit_TP = np.concatenate([x_fit_neg_TP] + [x_fit_pos_TP])
        x_fit_TN = np.concatenate([x_fit_pos_TN] + [x_fit_neg_TN])

        params_pos_TP, extras_pos_TP = opt.curve_fit(erfunc, x_fit_pos_TP, y_fit_pos_TP)
        params_neg_TP, extras_neg_TP = opt.curve_fit(
            erfunc, x_fit_neg_TP, y_fit_neg_TP, p0=[-4000, 0, 100]
        )
        params_pos_TN, extras_pos_TN = opt.curve_fit(erfunc, x_fit_pos_TN, y_fit_pos_TN)
        params_neg_TN, extras_neg_TN = opt.curve_fit(
            erfunc, x_fit_neg_TN, y_fit_neg_TN, p0=[-4000, 0, 100]
        )

        y_pos_TP = erfunc(x_fit_pos_TP, *params_pos_TP)
        y_neg_TP = erfunc(x_fit_neg_TP, *params_neg_TP)
        y_pos_TN = erfunc(x_fit_pos_TN, *params_pos_TN)
        y_neg_TN = erfunc(x_fit_neg_TN, *params_neg_TN)

        y_fit_TP = np.concatenate([y_neg_TP] + [y_pos_TP])
        y_fit_TN = np.concatenate([y_pos_TN] + [y_neg_TN])

        axs[0].plot(x_fit_TP, y_fit_TP, color="skyblue")
        axs[0].plot(x_fit_TN, y_fit_TN, color="purple")

        axs[1].plot(x_fit_TP, TestingPositive_HP - y_fit_TP, color="skyblue")
        axs[1].plot(x_fit_TN, TestingNegative_HP - y_fit_TN, color="purple")

        handles.extend([line1, line2, line3, line4])

        axs[1].legend(handles=handles, fontsize=13, framealpha=1, loc="lower right")

        # Add axis labels
        axs[0].set_xlabel("${\Delta}$displacement(${\mu}$m)")
        axs[0].set_ylabel("Force (N)")

        axs[1].set_xlabel("${\Delta}$displacement(${\mu}$m)")
        axs[1].set_ylabel("Residual (data-fit)")

        # Add grid and legend to plot
        axs[0].grid("-", lw=0.5, alpha=0.25)
        axs[1].grid("-", lw=0.5, alpha=0.25)

        axs[0].spines["top"].set_visible(False)
        axs[0].spines["right"].set_visible(False)
        axs[0].xaxis.set_ticks_position("bottom")
        axs[0].set_xlim(-500, 500)
        axs[0].set_ylim(-5000, 5000)
        axs[1].set_xlim(-500, 500)
        axs[1].set_ylim(-1000, 1000)
        axs[0].axhspan(comp_min, comp_max, color="grey", alpha=0.2)
        axs[0].axhspan(tens_min, tens_max, color="yellow", alpha=0.2)

        axs[0].axhline(y=0, color="k")
        axs[0].axvline(x=0, color="k")
        axs[1].axhline(y=0, color="k")
        axs[1].axvline(x=0, color="k")

        axs[0].text(-450, np.mean([comp_min, comp_max]), "Compression", weight="bold")
        axs[0].text(-450, np.mean([tens_min, tens_max]), "Tension", weight="bold")

        # Add title to plot
    title = (
        f"Individual Hard Point Breakaway Test at El:{El} deg, Az:{Az} deg\n"
        f"HP{hp_index + 1} - Block {block_n} -"
        f" {df.index[0].isoformat(timespec='seconds')[:-6]} -"
        f" {df.index[-1].isoformat(timespec='seconds')[:-6]}"
    )
    axs[0].text(0.6, 1.05, title, transform=axs[0].transAxes)

    # Save plot as png file
    os.makedirs("plots", exist_ok=True)
    filename = f"Force_displacement_fitting_residual_block_{block_n}_El_{El}_HP{hp_index + 1}.png"
    fig.savefig(f"plots/{filename}", dpi=150, bbox_inches="tight")
    fig.show()